In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [2]:
#사용자 아이디 입력
PI = -9010000000000000000



In [3]:
interaction_df = pd.read_csv('C:/Users/hahaj/tea/users_interactiontea.csv')

In [4]:
age_df = pd.read_csv('C:/Users/hahaj/tea/id_gender_age.csv')

In [5]:
interaction_df.head(5)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,gender,age(15~60)
0,1465413032,VIEW,a18,-8.850000e+18,1.260000e+18,NaN,NaN,NaN,male,18
1,1465412560,VIEW,a41,-1.030000e+18,3.620000e+18,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,female,56
2,1465416190,VIEW,a34,-1.130000e+18,2.630000e+18,NaN,NaN,NaN,male,39
3,1465413895,JJIM,a25,3.440000e+17,-3.170000e+18,NaN,NaN,NaN,male,55
4,1465412290,VIEW,a63,-4.450000e+17,5.610000e+18,NaN,NaN,NaN,male,45


In [6]:
age_df.head(5)

,personId,gender,age(15~60)
0,9200000000000000000,male,22
1,9140000000000000000,female,15
2,9110000000000000000,male,49
3,9010000000000000000,male,55
4,8970000000000000000,male,37


In [7]:
#interaction_df_over5 : users_interactiontea.csv파일에서 contentid에 개수를 count함.(interactCnt5이상인것만 추출: interation_df_over5)

interaction_df_over5 = (interaction_df
  .groupby('personId', group_keys=False)
  .apply(lambda df: df.assign(interactCnt = lambda d: d['contentId'].nunique()))
  .loc[lambda d: d['interactCnt'] >= 5]
)
interaction_df_over5.drop(['gender','age(15~60)'],axis = 1 , inplace = True)

In [8]:
#personid에 대한 고유값개수: 총 5회이상 액션(view, jjim, comment등)을 취한 등록된 아이디수

interaction_df_over5['personId'].nunique()

397

In [9]:
interaction_df_over5

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,interactCnt
866,1463138398,VIEW,a23,-9.220000e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5
1475,1463656314,VIEW,a1,-9.220000e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5
2916,1464113091,VIEW,a30,-9.220000e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5
5309,1462283851,VIEW,a66,-9.220000e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5
9221,1462452127,VIEW,a58,-9.220000e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5
...,...,...,...,...,...,...,...,...,...
5522,1464955959,VIEW,a57,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5
5566,1464955960,VIEW,a49,9.200000e+18,-9.110000e+18,NaN,NaN,NaN,5
5621,1464956193,VIEW,a36,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5
5623,1464955914,VIEW,a67,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5


In [10]:
merge_outer = pd.merge(interaction_df_over5,age_df, how='outer',on='personId')

#merge_outer 에 2개의 csv파일 취합

In [12]:
merge_outer #merge_outer 구조

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,interactCnt,gender,age(15~60)
0,1463138398,VIEW,a23,-9.220000e+18,-4.480000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
1,1463656314,VIEW,a1,-9.220000e+18,-7.820000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
2,1464113091,VIEW,a30,-9.220000e+18,-2.770000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
3,1462283851,VIEW,a66,-9.220000e+18,2.630000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
4,1462452127,VIEW,a58,-9.220000e+18,-3.670000e+18,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,5,male,29
...,...,...,...,...,...,...,...,...,...,...,...
9141,1464955959,VIEW,a57,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22
9142,1464955960,VIEW,a49,9.200000e+18,-9.110000e+18,NaN,NaN,NaN,5,male,22
9143,1464956193,VIEW,a36,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22
9144,1464955914,VIEW,a67,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22


In [13]:
# PersonId와 일치하는 값을 merge_outer에서 추출. IDGEN - gender, IDAGE - age 에 대해서 personid에 해당하는 값이다.

df=merge_outer[merge_outer['personId']==PI]

df = df.reset_index()
IDGEN=df.loc[0][10]
IDAGE=df.loc[0][11]
print(IDGEN ,IDAGE)

male 20


In [14]:
cond1 = merge_outer[(merge_outer['gender']==IDGEN) &(merge_outer['age(15~60)']<=IDAGE+5) & (merge_outer['age(15~60)']>=IDAGE-5)]
cond1

#cond1: merge_outer에서 같은 PI에 대해서 IDGEN 과 IDAGE( +- 5살) 에대한 해당 값만 가져옴.




,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,interactCnt,gender,age(15~60)
22,1464618134,VIEW,a56,-9.110000e+18,-5.490000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15
23,1465828659,VIEW,a19,-9.110000e+18,6.800000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15
24,1465828659,VIEW,a2,-9.110000e+18,6.800000e+18,NaN,NaN,NaN,7,male,15
25,1461580339,VIEW,a62,-9.110000e+18,-4.060000e+17,Mozilla/5.0 (Linux; Android 6.0; MotoG3 Build/...,SP,BR,7,male,15
26,1462468084,VIEW,a55,-9.110000e+18,5.480000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15
...,...,...,...,...,...,...,...,...,...,...,...
9141,1464955959,VIEW,a57,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22
9142,1464955960,VIEW,a49,9.200000e+18,-9.110000e+18,NaN,NaN,NaN,5,male,22
9143,1464956193,VIEW,a36,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22
9144,1464955914,VIEW,a67,9.200000e+18,-9.110000e+18,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,5,male,22


In [15]:
event_type_strength = {
    'VIEW': 1.0,
    'JJIM': 3.0, 
    'comment1' : 1.0,
    'comment2' : 2.0,
    'comment3' : 3.0,
    'comment4' : 4.0,
    'comment5' : 5.0
  
}

#액션에 대해서 평가 점수 선정



cond1['eventStrength'] = (
  cond1
    .loc[:, 'eventType']
    .apply(lambda d: event_type_strength[d])
)

#eventStrength열을 추가. 이 열에 eventType에 대한 값에 event_type_strength에 해당하는 값을 대입.

c:\users\hahaj\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [16]:
cond1.head(40)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,interactCnt,gender,age(15~60),eventStrength
22,1464618134,VIEW,a56,-9.110000e+18,-5.490000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15,1.0
23,1465828659,VIEW,a19,-9.110000e+18,6.800000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15,1.0
24,1465828659,VIEW,a2,-9.110000e+18,6.800000e+18,NaN,NaN,NaN,7,male,15,1.0
25,1461580339,VIEW,a62,-9.110000e+18,-4.060000e+17,Mozilla/5.0 (Linux; Android 6.0; MotoG3 Build/...,SP,BR,7,male,15,1.0
26,1462468084,VIEW,a55,-9.110000e+18,5.480000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15,1.0
27,1460748591,VIEW,a57,-9.110000e+18,-7.810000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15,1.0
28,1462965036,VIEW,a9,-9.110000e+18,2.920000e+18,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,7,male,15,1.0
29,1464096077,JJIM,a44,-9.060000e+18,-2.210000e+18,NaN,NaN,NaN,6,male,25,3.0
30,1464096055,VIEW,a26,-9.060000e+18,-2.210000e+18,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,6,male,25,1.0
31,1464979035,VIEW,a40,-9.060000e+18,7.790000e+18,NaN,NaN,NaN,6,male,25,1.0


In [17]:
interaction_full_df = (
  cond1
    .groupby(['personId', 'contentId'], as_index=False)['eventStrength']
    .sum()
    .assign(eventScore = lambda d: np.log10(1+d['eventStrength']))
)

#interaction_full_df 는 eventScore라는 열을 추가후 personID,contentId 를 그룹화후 이에 맞는  eventStrength 합에 대한 값을 대입
interaction_full_df.head(10)

,personId,contentId,eventStrength,eventScore
0,-9.110000e+18,a19,1.0,0.30103
1,-9.110000e+18,a2,1.0,0.30103
2,-9.110000e+18,a55,1.0,0.30103
3,-9.110000e+18,a56,1.0,0.30103
4,-9.110000e+18,a57,1.0,0.30103
5,-9.110000e+18,a62,1.0,0.30103
6,-9.110000e+18,a9,1.0,0.30103
7,-9.060000e+18,a23,3.0,0.60206
8,-9.060000e+18,a26,1.0,0.30103
9,-9.060000e+18,a40,1.0,0.30103


In [18]:
item_popularity = (interaction_full_df
 .groupby('contentId')['eventStrength'].sum()
 .sort_values(ascending=False)
 .reset_index()
)
#interaction_full_df에서 contentId별로 eventStrength를 합한 뒤, 내립차순으로 정렬(item_popularity - 연령별,성별별 인기도 순위)
item_popularity.head(10)

,contentId,eventStrength
0,a44,29.0
1,a17,29.0
2,a23,28.0
3,a11,27.0
4,a22,26.0
5,a52,26.0
6,a6,26.0
7,a61,24.0
8,a21,24.0
9,a1,24.0
